### Modified example from https://github.com/turboderp/exllamav2/blob/master/examples/inference.py - simple text input->text output stored in a variable

In [1]:
#!python --version

In [2]:
from exllamav2 import(
    ExLlamaV2,
    ExLlamaV2Config,
    ExLlamaV2Cache,
    ExLlamaV2Tokenizer,
    ExLlamaV2Cache_Q4,
)

from exllamav2.generator import (
    ExLlamaV2BaseGenerator,
    ExLlamaV2Sampler
)

import time

In [3]:
# Initialize model and cache

model_directory =  "/shared/analyst/models/Meta-Llama-3-70B-Instruct-4.0bpw-h6-exl2"#"/shared/analyst/models/Llama-3-8B-Instruct-262k-5.0bpw-h6-exl2"

config = ExLlamaV2Config(model_directory)
config.prepare()
config.max_seq_len = 32000
model = ExLlamaV2(config)
cache = ExLlamaV2Cache_Q4(model, lazy = True, max_seq_len=config.max_seq_len)
model.load_autosplit(cache)
tokenizer = ExLlamaV2Tokenizer(config)

# Initialize generator

generator = ExLlamaV2BaseGenerator(model, cache, tokenizer)

# Generate some text

settings = ExLlamaV2Sampler.Settings()
settings.temperature = 0.85
settings.top_k = 50
settings.top_p = 0.8
settings.token_repetition_penalty = 1.01
settings.disallow_tokens(tokenizer, [tokenizer.eos_token_id])

In [6]:
import ast
import textwrap

def get_docstring(node):
    """
    Extracts the docstring from an AST node representing a function definition.

    Args:
        node: An AST node representing a function definition.

    Returns:
        The docstring of the function as a string, or None if no docstring exists.
    """
    if isinstance(node.body[0], ast.Expr) and isinstance(node.body[0].value, ast.Str):
        return node.body[0].value.s.strip()
    return None

def get_function_body(source_lines, start_lineno, end_lineno):
    """
    Retrieves the function body (including comments) from source lines based on line numbers.

    Args:
        source_lines: A list of strings representing the lines of Python code.
        start_lineno: The line number (1-based) where the function definition starts.
        end_lineno: The line number (1-based) where the function definition ends.

    Returns:
        The function body as a single string, including any comments within the function.
    """
    return "\n".join(source_lines[start_lineno - 1:end_lineno])

def parse_python_file(file_path):
    """
    Parses a Python file, extracts functions, and gathers their docstrings and bodies.

    Args:
        file_path: The path to the Python file to be processed.

    Returns:
        A list of tuples. Each tuple contains the function name, its body, its docstring, 
        and its examples (if present in the docstring).
    """
    functions = []

    with open(file_path, "r") as file:
        source_lines = file.readlines()
        tree = ast.parse("".join(source_lines), filename=file_path)

    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            # Get the start and end line numbers of the function definition
            start_lineno = node.lineno
            end_lineno = node.body[-1].lineno if node.body else node.lineno

            # Grab any comments preceding the function definition
            docstring = get_docstring(node)
            if not docstring:
                continue

            # Get the function body including comments and docstrings
            function_body = get_function_body(source_lines, start_lineno, end_lineno)

            # Split docstring into description and examples
            description, _, examples = docstring.partition("Examples")

            # Remove extra leading/trailing whitespaces
            description = description.strip()

            # Format examples nicely
            examples = textwrap.dedent(examples.strip())

            function_split = function_body.split('"""')
            if len(function_split) > 2:
                function_body = function_split[0] + function_split[2]
            description_split = description.split("Examples\n--------")
            description = description_split[0]

            functions.append((node.name, function_body, description, examples))

    return functions
def get_specific_function(functions, desired_function):
    """
    Finds a specific function from a list based on its name.

    Args:
        functions: A list of tuples containing function information (name, body, docstring, examples).
        desired_function: The name of the function to search for.

    Returns:
        A tuple containing the function information (name, body, docstring, examples) 
        if the function is found, or "Unable to Find" if not found.
    """
    for func_name, function_body, description, examples in functions:
        if func_name == desired_function:
            return func_name, function_body, description, examples
    return "Unable to Find"
def generate_prompt(function1, function2, function3, unknown_function):
    """
    Constructs a formatted string as a prompt for a large language model. 

    This function combines information about three known functions and one unknown function 
    into a single string formatted for consumption by a large language model. 

    Args:
        function1, function2, function3: Tuples containing information (name, body, docstring, examples) 
        for three reference functions used as examples.
        unknown_function: A tuple containing information (name, body, docstring, examples) 
        for the function where examples need to be generated.

    Returns:
        A string containing the formatted prompt for the large language model. 
        The prompt includes information about all four functions (name, body, docstring, and examples).
    """
    prompt = "Function: " + function1[0] + "\nFunction Body: " + function1[1] + "\nDocstrings: "+ function1[2] + "\nExamples: "+ function1[3]
    prompt += "\nFunction: " + function2[0] + "\nFunction Body: " + function2[1] + "\nDocstrings: "+ function2[2] + "\nExamples: "+ function2[3]
    prompt += "\nFunction: " + function3[0] + "\nFunction Body: " + function3[1] + "\nDocstrings: "+ function3[2] + "\nExamples: "+ function3[3]
    prompt += "\nFunction: " + unknown_function[0] + "\nFunction Body: " + unknown_function[1] + "\nDocstrings: "+ unknown_function[2] + "\nExamples: ------"
    return prompt    
def write_to_file(filename, string):
    """
    Writes a string to a file, appending it to the end and adding formatting markers.

    This function opens a file in append mode (`'a'`) and writes the provided string to it. 
    It prepends "Assistant" to the string and adds additional formatting lines for better 
    organization within the file.

    Args:
        filename (str): The path to the file where the string will be written.
        string (str): The string to be written to the file.

    Returns:
        None
    """
    with open(filename, 'a') as file:
        file.write("Assistant" + string + "\n\n")
        file.write("-------------------------------------------------------------------------------------------------\n\n")
    print("finished")

def get_examples(functions):
    """
    Generates example code for a provided function using a large language model.

    This function prepares a training prompt for a large language model to generate code examples 
    for a function named in the `functions` argument. 
    It extracts known functions from two Python source files and constructs a prompt highlighting 
    their functionalities (names, bodies, docstrings, and examples).

    Args:
        functions (List[str]): A list of functions.

    Returns:
        None: The function doesn't explicitly return anything, but it writes generated examples 
              to a file named "examples_testing_70.txt".
    """
    file_path = "_linalg.py"
    lin_functions = parse_python_file(file_path)
    
    function1 = get_specific_function(lin_functions, "inv")
    function2 = get_specific_function(lin_functions, "det")
    function3 = get_specific_function(lin_functions, "eigvals")

    file_path = "core.py"
    
    core_functions = parse_python_file(file_path)
    
    function4 = get_specific_function(core_functions, "minimum_fill_value")
    function5 = get_specific_function(core_functions, "asarray")
    
    for function in functions:
        unknown_function = function
        unknown_function_name = function[0]
        prompt = generate_prompt(function1, function3, function5, unknown_function)
    
        system_prompt = "You take in three example functions with sections Function, Function Body, Docstrings, and Examples. "
        system_prompt += "You are given a fourth function, function body, and docstring. " 
        system_prompt += f"Please write examples for the {unknown_function_name} function. " 
        system_prompt += f"Provide only examples for the {unknown_function_name} function. "
        system_prompt += "Do not provide repeat examples. "
        system_prompt += "Do not start an example without finishing it. "
        
        texts = [f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|>\n"]
        texts.append(f'<|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>')
        rendered_prompt = ''.join(texts)

        rendered_prompt
        tokenizer.encode(rendered_prompt)
        tokenizer.encode(rendered_prompt).shape[-1]

        max_new_tokens = 512

        generator.warmup()
        time_begin = time.time()

        output = generator.generate_simple(rendered_prompt, settings, max_new_tokens, seed = 1234)

        time_end = time.time()
        time_total = time_end - time_begin

        #print(output)
        #print()
        print(f"Response generated in {time_total:.2f} seconds, {max_new_tokens} tokens, {max_new_tokens / time_total:.2f} tokens/second")
        assistant_texts = output.split("assistant")
        write_to_file("examples_testing_70.txt", assistant_texts[1])

In [ ]:
# Example Usage
# This example generates examples for svdvals, cond, matrix_rank, and eigh.
file_path = "_linalg.py"
functions = parse_python_file(file_path)
list_of_functions = []
list_of_functions.append(get_specific_function(functions, "svdvals"))
list_of_functions.append(get_specific_function(functions, "cond"))
list_of_functions.append(get_specific_function(functions, "matrix_rank"))
list_of_functions.append(get_specific_function(functions, "eigh"))
get_examples(list_of_functions)

Response generated in 165.27 seconds, 512 tokens, 3.10 tokens/second
finished
Response generated in 175.89 seconds, 512 tokens, 2.91 tokens/second
finished
Response generated in 183.31 seconds, 512 tokens, 2.79 tokens/second
finished
